<a href="https://colab.research.google.com/github/Bprs68/Rossmann-Sales-Prediction-/blob/main/Random_Forest_Hyperparameter_Tuning_Notebook_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder


import warnings    
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
file_path = '/content/drive/MyDrive/AlmaBetter/Capstone Projects/Capstone Project - Supervised Machine Learning- Regression/cleaned_data/'

In [5]:
train_df = pd.read_csv(file_path+'train.csv')

In [6]:
test_df = pd.read_csv(file_path+'test.csv')

In [7]:
#setting date and store as index
train_df.set_index(['Date','Store'], inplace=True)
train_df.sort_values(by=['Date','Store'], inplace=True)

test_df.set_index(['Date','Store'], inplace=True)
test_df.sort_values(by=['Date','Store'], inplace=True)

In [8]:
train_df.head(1)

,,DayOfWeek,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Year,Month,WeekOfYear,DayOfYear,CompetitionOpen,Promo2Open,Promo2running
Date,Store,,,,,,,,,,,,,,,,,
2013-01-01,85,2,8.34759,619,0,1,1,b,a,1870.0,0,2013,1,1,1,15.0,0.0,0


In [9]:
train_df.columns

Index(['DayOfWeek', 'Sales', 'Customers', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'Promo2', 'Year', 'Month', 'WeekOfYear', 'DayOfYear', 'CompetitionOpen',
       'Promo2Open', 'Promo2running'],
      dtype='object')

In [10]:
test_df.head(1)

,,DayOfWeek,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Year,Month,WeekOfYear,DayOfYear,CompetitionOpen,Promo2Open,Promo2running
Date,Store,,,,,,,,,,,,,,,,,
2015-06-15,1,1,8.615771,586,1,0,0,c,a,1270.0,0,2015,6,25,166,81.0,0.0,0


## Test-Train-Split

In [11]:
# dividing train df into train_x and train_y 
X_train = train_df.drop('Sales', axis=1)
y_train = train_df[['Sales']]

#dividing test df into test_x and test_y
X_test = test_df.drop("Sales", axis=1)
y_test = test_df[['Sales']]

One Hot Encoding of categorical columns to conver them into numerical columns

In [12]:
# making a list of categorical columns
Categorical_columns = ['DayOfWeek','StoreType','Assortment']

In [13]:
# assigning one hot encoder
enc = OneHotEncoder(sparse=False)

In [14]:
enc.fit(X_train[Categorical_columns])

OneHotEncoder(sparse=False)

In [15]:
# getting a new list of encoded columns from Categorical_columns in train data
encoded_cols = enc.get_feature_names(Categorical_columns).tolist()

In [16]:
encoded_cols

['DayOfWeek_1',
 'DayOfWeek_2',
 'DayOfWeek_3',
 'DayOfWeek_4',
 'DayOfWeek_5',
 'DayOfWeek_6',
 'DayOfWeek_7',
 'StoreType_a',
 'StoreType_b',
 'StoreType_c',
 'StoreType_d',
 'Assortment_a',
 'Assortment_b',
 'Assortment_c']

In [17]:
#performing transformation on training data and creating new features for encoded columns
X_train[encoded_cols] = enc.transform(X_train[Categorical_columns])

In [18]:
#dropping old categorical columns from training data
X_train.drop(Categorical_columns, axis=1, inplace=True)

In [19]:
#performing transformation on testing data and creating new features for encoded columns
X_test[encoded_cols] = enc.transform(X_test[Categorical_columns])

In [20]:
#dropping old categorical columns from testing data
X_test.drop(Categorical_columns, axis=1, inplace=True)

In [21]:
X_train.columns

Index(['Customers', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'CompetitionDistance', 'Promo2', 'Year', 'Month', 'WeekOfYear',
       'DayOfYear', 'CompetitionOpen', 'Promo2Open', 'Promo2running',
       'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4',
       'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7', 'StoreType_a',
       'StoreType_b', 'StoreType_c', 'StoreType_d', 'Assortment_a',
       'Assortment_b', 'Assortment_c'],
      dtype='object')

### **Time for transformations in our data**

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train[list(X_train.columns)] = scaler.fit_transform(X_train[list(X_train.columns)])
X_test[list(X_test.columns)] = scaler.fit_transform(X_test[list(X_test.columns)])

scaler = StandardScaler()
y_train[list(y_train.columns)] = scaler.fit_transform(y_train[list(y_train.columns)])
y_test[list(y_test.columns)] = scaler.transform(y_test[list(y_train.columns)])

In [ ]:
# importing evaluation matrices
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score



As expected high bias and high variance.

**Hyperparameter Tuning**

In [23]:
# importing random forest from ScikitLearn
from sklearn.ensemble import RandomForestRegressor

In [24]:
from sklearn.model_selection import RandomizedSearchCV

In [25]:
rf_tuned = RandomForestRegressor(random_state=68)

In [ ]:
from scipy.stats import randint

rs_space={'max_depth':[100,150,None],
              'n_estimators':[100,150,200],
              'max_features':['auto','sqrt'],
              'min_samples_leaf':randint(1,4),
              'min_samples_split':np.arange(2, 10, step=2)
         }

In [26]:
rf_grid = RandomizedSearchCV(estimator = rf_tuned, param_distributions = rs_space, n_iter = 10, cv = 3, verbose=2, random_state=68)

model_random = rf_grid.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=6, n_estimators=100; total time= 1.9min
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=6, n_estimators=100; total time= 1.8min
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=6, n_estimators=100; total time= 1.7min
[CV] END max_depth=100, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=100; total time= 6.8min
[CV] END max_depth=100, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=100; total time= 6.8min
[CV] END max_depth=100, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=100; total time= 6.7min
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=100; total time= 6.7min
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_e

In [1]:
# getting best parameters
model_random.best_params_

NameError: ignored

In [31]:
# assigning to variable
params = model_random.best_params_

In [28]:
# assigning variable with tuned params
rf_tuned = RandomForestRegressor(max_depth = None, max_features = 'auto', min_samples_leaf=1, n_estimators=150)

In [29]:
# fitting on training data
rf_tuned.fit(X_train,y_train)

RandomForestRegressor(n_estimators=150)

In [32]:
# predicting train_y
y_pred_train_rft = rf_tuned.predict(X_train)

In [33]:
# predicting test_y
y_pred_test_rft = rf_tuned.predict(X_test)

In [35]:
# importing evaluation matrices
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [36]:
# printing evaluation matrices for our model
print(f'r2_score for training data is {r2_score(y_pred_train_rft,y_train)}')
print(f'r2_score for testing data is {r2_score(y_pred_test_rft,y_test)}')

print(f'Adjusted r2_score for training data is {round(1 - (1-r2_score(y_train, y_pred_train_rft)) * (len(y_train)-1)/(len(y_train)-X_train.shape[1]-1),6)}')
print(f'Adjusted r2_score for testing data is {round(1 - (1-r2_score(y_test, y_pred_test_rft)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1),6)}')

print(f'Mean Absolute Error for training data is {mean_absolute_error(y_pred_train_rft,y_train)}')
print(f'Mean Absolute Error for testing data is {mean_absolute_error(y_pred_test_rft,y_test)}')

print(f'Mean Squared Error for training data is {mean_squared_error(y_pred_train_rft,y_train)}')
print(f'Mean Squared Error for testing data is {mean_squared_error(y_pred_test_rft,y_test)}')

r2_score for training data is 0.9964959757511653
r2_score for testing data is 0.9510617249214244
Adjusted r2_score for training data is 0.996559
Adjusted r2_score for testing data is 0.952459
Mean Absolute Error for training data is 0.0430367119650388
Mean Absolute Error for testing data is 0.1536215321612703
Mean Squared Error for training data is 0.003440781546560589
Mean Squared Error for testing data is 0.04475569627990811
